<a href="https://colab.research.google.com/github/skyfuryonline/semeval/blob/master/latest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
! pip install datasets

In [3]:
from datasets import Dataset
from transformers import AutoTokenizer
from transformers import T5Tokenizer

# 示例数据
# data = {
#         "source_locale": ["en","en"],
#         "target_locale": ["fr","fr"],
#         "source": ["How many Star Wars movies has Chewbacca appeared in?","'Good!' said Norbert, 'there's one of my father's spies gone; that leaves only the little cripple Napier.'"],
#         "target": ["Combien de films Star Wars Chewbacca est-il apparu?",'– Bon! dit Norbert, voilà un des espions de mon père parti ; il ne reste plus que le petit boiteux Napier.'],
# }
from datasets import load_dataset

data = load_dataset("opus_books", "en-fr")
data = data['train'].train_test_split(test_size=0.2)

train_data = {
    "source_locale": [],
    "target_locale": [],
    "source":[],
    "target":[],
}
test_data = {
    "source_locale": [],
    "target_locale": [],
    "source":[],
    "target":[],
}

cnt = 1
for item in data['train']:
  for lang,sentence in item['translation'].items():
    if cnt %2 != 0:
      train_data['source_locale'].append(lang)
      train_data['source'].append(sentence)
    else:
      train_data['target_locale'].append(lang)
      train_data['target'].append(sentence)
    cnt += 1

cnt = 1
for item in data['test']:
  for lang,sentence in item['translation'].items():
    if cnt %2 != 0:
      test_data['source_locale'].append(lang)
      test_data['source'].append(sentence)
    else:
      test_data['target_locale'].append(lang)
      test_data['target'].append(sentence)
    cnt += 1



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [4]:
# 创建Dataset对象
train_data = Dataset.from_dict(train_data)
test_data = Dataset.from_dict(test_data)
print(train_data[0])
print(test_data[0])

{'source_locale': 'en', 'target_locale': 'fr', 'source': 'At 10:50 in the evening, that electric light reappeared three miles to windward of the frigate, just as clear and intense as the night before.', 'target': 'A dix heures cinquante minutes du soir, la clarté électrique réapparut, à trois milles au vent de la frégate, aussi pure, aussi intense que pendant la nuit dernière.'}
{'source_locale': 'en', 'target_locale': 'fr', 'source': '"Mr. John?"', 'target': '-- M. John?'}


In [5]:
column_name = train_data.column_names
print(column_name)
# 加载 T5 模型的分词器
model_id = "google-t5/t5-small"  # 使用 T5-small 模型
# tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer = T5Tokenizer.from_pretrained(model_id)

['source_locale', 'target_locale', 'source', 'target']


You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [6]:
# 任务说明和输入格式
prefix = "translate English to French: "

# 预处理函数
def preprocess_function(examples):
    # 为每个句子添加任务描述前缀
    inputs = [prefix + example for example in examples['source']]
    targets = [example for example in examples['target']]
    # 使用 T5 分词器对输入和目标进行编码
    model_inputs = tokenizer(inputs, text_target=targets, max_length=64, truncation=True, padding="max_length")
    return model_inputs

# 预处理数据
tokenized_train = train_data.map(preprocess_function, batched=True,remove_columns=column_name)
tokenized_test = test_data.map(preprocess_function,batched=True,remove_columns=column_name)
# 查看预处理后的数据
print(tokenized_train[0])
print(tokenized_test[0])

Map:   0%|          | 0/101668 [00:00<?, ? examples/s]

Map:   0%|          | 0/25417 [00:00<?, ? examples/s]

{'input_ids': [13959, 1566, 12, 2379, 10, 486, 335, 10, 1752, 16, 8, 2272, 6, 24, 2806, 659, 22993, 855, 9, 1271, 386, 2286, 12, 2943, 2239, 13, 8, 19791, 342, 6, 131, 38, 964, 11, 6258, 38, 8, 706, 274, 5, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'labels': [71, 20679, 8446, 16109, 76, 1841, 676, 146, 12514, 6, 50, 6860, 2229, 13437, 1417, 10198, 76, 17, 6, 3, 85, 5611, 3293, 15, 7, 185, 9370, 20, 50, 2515, 154, 5339, 6, 1292, 4621, 6, 1292, 6258, 238, 4530, 50, 12266, 6062, 5, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}
{'input_ids': [13959, 1566, 12, 2379, 10, 96, 329, 52, 5, 1079, 4609, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [7]:
from transformers import DataCollatorForSeq2Seq
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model_id)

In [8]:
! pip install evaluate
! pip install sacrebleu
import evaluate
metric = evaluate.load("sacrebleu")

In [22]:
import numpy as np


def postprocess_text(preds, labels):
   # 确保 preds 和 labels 是列表，并去除空白字符
    preds = [pred.strip() for pred in preds]
    # BLEU 需要 label 是嵌套列表
    labels = [[label.strip()] for label in labels]

    return preds, labels


def compute_metrics(eval_preds):
    preds, labels = eval_preds
    # preds通常是三维？(batch_size,seq_len,vocab_size)
    # 维度不对
    # print(len(preds))
    # # 2--batch_size
    # print(len(preds[0]))
    # # 50--
    # print(len(preds[1]))
    # # 50

    # print(len(labels))
    # 50
    # print(len(labels[0]))
    # 64
    # print(len(labels[1]))
    # 64
    # ...



    if isinstance(preds, tuple):
        preds = preds[0]
    # 卡在这步
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)

    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)


    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

In [10]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(model_id)

In [23]:

tokenized_train_slice =Dataset.from_dict(tokenized_train[0:50])
tokenized_test_slice = Dataset.from_dict(tokenized_test[0:50])
'''
{
  'input_ids': [
    [13959, 1566, 12, 2379, 10, 363, 3, 9, 9529, 47, 48, 21, 2714, 8158, 55, 113, 133, 4403, 120, 43, 2767, 190, 8, 296, 406, 19960, 24, 78, 231, 25972, 655, 16415, 16, 8, 829, 1964, 13, 25575, 6, 38, 47, 270, 4759, 16, 80, 928, 5, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [13959, 1566, 12, 2379, 10, 37, 1591, 4659, 1120, 7886, 3834, 38, 34, 2301, 550, 45, 8, 3, 7, 232, 2667, 7, 57, 8, 9866, 6, 11, 62, 1116, 4363, 44, 128, 307, 6, 2943, 53, 26462, 7, 6, 3, 18091, 10856, 13704, 24, 2225, 178, 12, 8147, 385, 57, 385, 117, 68, 62, 141, 12, 24735, 20917, 120, 16, 8, 3, 12342, 1],
    [13959, 1566, 12, 2379, 10, 299, 1919, 163, 2012, 70, 13608, 6, 24, 1919, 84, 255, 141, 5327, 437, 160, 166, 14295, 6, 11, 213, 255, 47, 88, 26, 550, 160, 19429, 159, 51, 16, 3, 9, 471, 6, 2852, 3, 9, 2335, 13, 2934, 44, 728, 6, 11, 13049, 6257, 139, 66, 8, 10680, 5575, 40, 4664, 13, 8, 1059, 5, 1, 0],
    [13959, 1566, 12, 2379, 10, 4589, 12, 2, 2528, 3623, 12, 8, 12530, 93, 3351, 29, 3167, 6, 11, 22374, 21, 8, 891, 115, 154, 20, 2788, 15, 2, 29838, 6, 13, 3351, 7, 535, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [13959, 1566, 12, 2379, 10, 695, 260, 6761, 2236, 6, 3, 173, 3, 2246, 17, 759, 18, 7170, 90, 3, 7, 9, 1999, 546, 22, 173, 8757, 9, 185, 10490, 393, 7632, 2970, 5, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [13959, 1566, 12, 2379, 10, 216, 141, 12, 65, 324, 78, 38, 59, 12, 2615, 6398, 13, 8, 659, 117, 163, 213, 8, 861, 2804, 28, 4226, 6, 28, 8, 3, 15956, 15, 655, 13, 3, 9, 30890, 6, 3, 88, 228, 59, 26537, 190, 406, 3, 9052, 4890, 112, 3, 9577, 7, 5, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
    ],
  'attention_mask': [
    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0],
    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
    ],
  'labels': [
    [12281, 4897, 171, 50, 24380, 8158, 285, 3, 12463, 6703, 1211, 76, 90, 2921, 21072, 1532, 3, 7, 22, 6487, 49, 546, 22, 173, 3223, 1439, 17, 247, 2633, 3, 40, 22, 12450, 1257, 11785, 20, 9691, 75, 1238, 546, 22, 693, 3, 35, 24870, 9, 155, 3, 35, 197, 798, 247, 73, 7224, 3, 6814, 3, 55, 1, 0, 0, 0, 0, 0],
    [312, 9467, 3, 7, 31, 154, 109, 20676, 11743, 297, 3, 35, 3, 7, 31, 29041, 288, 146, 8318, 9, 159, 93, 8882, 17, 7, 6, 3, 15, 17, 678, 283, 2687, 21707, 25776, 7, 3, 85, 93, 10785, 15, 7, 12280, 7, 3, 15, 17, 3731, 25977, 7, 6, 12421, 7, 15941, 8901, 7, 285, 2933, 17, 5635, 20, 3, 7, 31, 1],
    [3307, 1919, 20, 20676, 3, 4933, 624, 50, 25587, 257, 6, 197, 1919, 546, 31, 693, 11489, 9, 155, 2873, 3, 7, 9, 13539, 10964, 3890, 15, 6, 3, 15, 17, 407, 3, 693, 142, 3, 22288, 3, 35, 3, 3464, 17, 4520, 20, 3, 7, 9, 7985, 6, 3, 7603, 122, 1841, 3, 26, 31, 202, 4897, 6, 8757, 721, 3, 9, 1],
    [27889, 20, 7484, 185, 3, 13878, 159, 93, 3351, 29, 3167, 6, 3, 15, 17, 4399, 172, 3, 40, 22, 12982, 154, 20, 2788, 18, 29838, 20, 3351, 7, 5, 1168, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [282, 3, 88, 5468, 3, 88, 2448, 808, 1520, 13, 8, 3, 7, 9, 1999, 84, 3, 88, 3, 89, 6521, 46, 3496, 120, 550, 5, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [802, 146, 17, 142, 3, 30020, 52, 171, 3, 29, 15, 330, 23013, 20, 7207, 50, 18734, 49, 15, 6, 3, 173, 142, 8757, 9, 247, 50, 23289, 3, 22206, 117, 7885, 6, 407, 3, 40, 31, 7776, 330, 8870, 3, 9, 3, 40, 31, 8284, 6, 393, 3, 7, 9, 5759, 924, 15, 20, 30890, 6, 759, 3, 29, 15, 20914, 17, 1]
    ]}
'''

training_args = Seq2SeqTrainingArguments(
    output_dir="my_awesome_opus_books_model",
    evaluation_strategy="steps",
    logging_steps=2,

    learning_rate=2e-5,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    weight_decay=0.01,
    # save_total_limit=3,
    num_train_epochs=10,
    # predict_with_generate=True,
    fp16=True,
    # push_to_hub=True,
    report_to ="none",
    gradient_checkpointing=True,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    # train_dataset=tokenized_train,
    # eval_dataset=tokenized_test,
    train_dataset=tokenized_train_slice,
    eval_dataset=tokenized_test_slice,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-23-544b4dc50488>:49: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


Step,Training Loss,Validation Loss


50
64
64
64


TypeError: int() argument must be a string, a bytes-like object or a real number, not 'list'

In [ ]:
text = "translate English to French: hello"

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_id)
inputs = tokenizer(text, return_tensors="pt").input_ids
inputs = inputs.to("cuda")

In [ ]:
from transformers import AutoModelForSeq2SeqLM
outputs = model.generate(inputs, max_new_tokens=40, do_sample=True, top_k=30, top_p=0.95)
tokenizer.decode(outputs[0], skip_special_tokens=True)